<a href="https://colab.research.google.com/github/hellwalker3/naive-txt2img-api-of-sd/blob/main/naive_text2img_api_client_sample_(LoRA%2Bcontrolnet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title URL of API server
api_url = 

In [ ]:
#@title Image encode and decode functions
import io
from io import BytesIO
import base64
from PIL import Image, PngImagePlugin
import cv2

def encode_array_to_base64(img):
    retval, buffer = cv2.imencode('.jpg', img)
    b64img = base64.b64encode(buffer).decode("utf-8")
    return b64img

def decode_base64_to_image(encoding):
    if encoding.startswith("data:image/"):
        encoding = encoding.split(";")[1].split(",")[1]
    try:
        image = Image.open(BytesIO(base64.b64decode(encoding)))
        return image
    except Exception as err:
        raise HTTPException(status_code=500, detail="Invalid encoded image")

def encode_pil_to_base64(image):
    with io.BytesIO() as output_bytes:
        use_metadata = False
        metadata = PngImagePlugin.PngInfo()
        for key, value in image.info.items():
            if isinstance(key, str) and isinstance(value, str):
                metadata.add_text(key, value)
                use_metadata = True
        image.save(output_bytes, format="PNG", pnginfo=(metadata if use_metadata else None), quality=95)

        bytes_data = output_bytes.getvalue()
    return 'data:image/png;base64,' + str(base64.b64encode(bytes_data), 'utf-8')


In [ ]:
# @title Reflect arguments passed to txt2img.: Instead of passing the scripts.external_code.ControlNetUnit object, use the format [enabled, module, model, weight, image, guidance_end] for the arguments. When passing controlnet images, use the following format: {'image': encode_arry_to_base64(cv2.imread(img_path)), 'mask': None}
def get_api_arguments(prompt, seed, control_pose_path, lora_name):
    # sample arguments (LoRA+t2iadapter_keypose)
    input_dic={'id_task': 'task(lq2abqt0nqbc7pi)',
     'prompt': prompt,
     'negative_prompt': """
     EasyNegative, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts,signature, watermark, username, blurry, artist name
     """,
     'prompt_styles': [],
     'steps': 28,
     'sampler_index': 0,
     'restore_faces': False,
     'tiling': False,
     'n_iter': 1,
     'batch_size': 1,
     'cfg_scale': 12,
     'seed': seed,
     'subseed': -1.0,
     'subseed_strength': 0,
     'seed_resize_from_h': 0,
     'seed_resize_from_w': 0,
     'seed_enable_extras': False,
     'height': 512,
     'width': 512,
     'enable_hr': False,
     'denoising_strength': 0.7,
     'hr_scale': 2,
     'hr_upscaler': 'Latent',
     'hr_second_pass_steps': 0,
     'hr_resize_x': 0,
     'hr_resize_y': 0,
     'override_settings_texts': []}
    input_dic['args'] = (0, True, False, 'LoRA', lora_name, 0.65, 0.65, 'LoRA', 'None', 1, 1, 'LoRA', 'None', 1, 1, 'LoRA', 'None', 1, 1, 'LoRA', 'None', 1, 1, None, 'Refresh models',
                        [True,  'none', 't2iadapter_keypose_sd14v1 [ba1d909a]', 1, {'image': encode_array_to_base64(cv2.imread(control_pose_path)), 'mask': None}, 1],
                         False, False, 'positive', 'comma', 0, False, False, '', 1, '', 0, '', 0, '', True, False, False, False, 0, None, None, 50)
    return input_dic


In [ ]:
# @title Sample code to generate sequential images for GIF using LoRA and t2iadapter_keypose
import requests, os, time

start_idx = 1
end_idx = 10
control_pose_base_path = "path/to/control_pose_images"
output_base_path = "path/to/output"
prompt = "Your prompt text"
seed = 42
lora_name = "your_lora_name"

os.makedirs(output_base_path, exist_ok=True)
for i in range(start_idx, end_idx + 1, 1):
    start_time = time.time()
    input_dic = get_api_arguments(prompt, seed, f'{control_pose_base_path}/img{i}.png', lora_name)

    # get generated image
    try:
        response = requests.post(url=f'{api_url}/sdapi/v1/txt2img2', json=input_dic)
        r = response.json()
    except:
        time.sleep(10)
        response = requests.post(url=f'{api_url}/sdapi/v1/txt2img2', json=input_dic)
        r = response.json()

    im = r['images'][0]
    image = Image.open(io.BytesIO(base64.b64decode(im.split(",", 1)[1])))

   
    png_payload = {
        "image": "data:image/png;base64," + im
    }

    # get png info
    try:
        response2 = requests.post(url=f'{api_url}/sdapi/v1/png-info', json=png_payload)
        pnginfo = PngImagePlugin.PngInfo()
        pnginfo.add_text("parameters", response2.json().get("info"))
    except:
        time.sleep(10)
        response2 = requests.post(url=f'{api_url}/sdapi/v1/png-info', json=png_payload)
        pnginfo = PngImagePlugin.PngInfo()
        pnginfo.add_text("parameters", response2.json().get("info"))

    # save image with png info
    image.save(f'{output_base_path}/img{i}.png', pnginfo=pnginfo)
    print(i, time.time() - start_time)
